# Nearest Neighbors

When exploring a large set of documents -- such as Wikipedia, news articles, StackOverflow, etc. -- it can be useful to get a list of related material. To find relevant documents you typically
1. Decide on a notion of similarity
* Find the documents that are most similar 

The goal fo this notebook is to:
* Gain intuition for different notions of similarity and practice finding similar documents. 
* Explore the tradeoffs with representing documents using raw word counts and TF-IDF
* Explore the behavior of different distance metrics by looking at the Wikipedia pages most similar to President Obama’s page.

**Note to Amazon EC2 users**: To conserve memory, make sure to stop all the other notebooks before running this notebook.

## Import necessary packages

As usual we need to first import the Python packages that we will need.

In [89]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from scipy.sparse import csr_matrix  
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
%matplotlib inline

## Load Wikipedia dataset

We will be using the same dataset of Wikipedia pages that we used in the Machine Learning Foundations course (Course 1). Each element of the dataset consists of a link to the wikipedia article, the name of the person, and the text of the article (in lowercase).  

In [2]:
wiki = pd.read_csv('people_wiki.csv')

In [3]:
print(wiki.shape)
wiki.head()

(59071, 3)


,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


## Extract word count vectors

Extract word count vectors using sklearn function.  We add this as a column in `wiki`.

In [4]:
# define a function to do word cuont
def count_word(words):
    counts = dict()
    for word in words.split():
        counts[word] = counts.get(word, 0) + 1
    return counts

In [5]:
# use collections.Counter to do word count:
from collections import Counter
def count_words(x): 
    return dict(Counter(x.split()))

wiki['word_count'] = map( count_words, wiki['text'])
wiki.head()

,URI,name,text,word_count
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,"{u'selection': 1, u'carltons': 1, u'being': 1,..."
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,"{u'precise': 1, u'thomas': 1, u'closely': 1, u..."
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,"{u'just': 1, u'issued': 1, u'mainly': 1, u'nom..."
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,"{u'englishreading': 1, u'all': 1, u'bauforschu..."
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,"{u'own': 2, u'they': 1, u'gangstergenka': 1, u..."


## Find nearest neighbors

Let's start by finding the nearest neighbors of the Barack Obama page using the word count vectors to represent the articles and Euclidean distance to measure distance.  For this, again will we use a sklearn implementation of nearest neighbor search.
Let's look at the top 10 nearest neighbors by performing the following query:

### 1. Use CountVectorizer() to turn text to columns of word count
One way is to use the CountVectorizer to create a sparse matrix of word count with one column for each word:

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
wordcount_vec = vectorizer.fit_transform(wiki['text'])

feature_names = vectorizer.get_feature_names()
print(wordcount_vec.shape, len(feature_names) )

((59071, 548429), 548429)


In [7]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric='euclidean', algorithm='brute')

OB_text = vectorizer.transform(wiki.loc[ wiki['name']=='Barack Obama', 'text'])
distance, index = model.kneighbors(X =OB_text, n_neighbors=11) 

pd.concat([wiki[['name']].iloc[index[0].tolist()].reset_index().drop('index', axis=1) ,
           pd.Series(distance[0])] , axis=1 )

,name,0
0,Barack Obama,0.000000
1,Joe Biden,33.015148
2,George W. Bush,34.307434
3,Mitt Romney,35.791060
4,Lawrence Summers,36.069378
5,Walter Mondale,36.249138
6,Francisco Barrio,36.276714
7,Don Bonker,36.400549
8,Wynn Normington Hugh-Jones,36.441734
9,Refael (Rafi) Benvenisti,36.837481


### 2. Use DictVectorizer()  to turn 'word_count' to columns of word count¶

In [8]:
DV = DictVectorizer()
wordcount_vec2 = DV.fit_transform(wiki['word_count'])
OB_text = DV.transform(wiki.loc[ wiki['name']=='Barack Obama', 'word_count'])
OB_text

<1x548561 sparse matrix of type '<type 'numpy.float64'>'
	with 273 stored elements in Compressed Sparse Row format>

In [9]:
model2 = NearestNeighbors( n_neighbors= 10).fit(wordcount_vec2) 
distance2, index2 = model2.kneighbors(X =OB_text, n_neighbors=11) 

pd.concat([wiki[['name']].iloc[index2[0].tolist()].reset_index().drop('index', axis=1) ,
           pd.Series(distance2[0])] , axis=1 )

,name,0
0,Barack Obama,0.000000
1,Joe Biden,33.075671
2,George W. Bush,34.394767
3,Lawrence Summers,36.152455
4,Mitt Romney,36.166283
5,Francisco Barrio,36.331804
6,Walter Mondale,36.400549
7,Wynn Normington Hugh-Jones,36.496575
8,Don Bonker,36.633318
9,Andy Anstett,36.959437


All of the 10 people are politicians, but about half of them have rather tenuous connections with Obama, other than the fact that they are politicians.

* Francisco Barrio is a Mexican politician, and a former governor of Chihuahua.
* Walter Mondale and Don Bonker are Democrats who made their career in late 1970s.
* Wynn Normington Hugh-Jones is a former British diplomat and Liberal Party official.
* Andy Anstett is a former politician in Manitoba, Canada.

Nearest neighbors with raw word counts got some things right, showing all politicians in the query result, but missed finer and important details.

For instance, let's find out why Francisco Barrio was considered a close neighbor of Obama.  To do this, let's look at the most frequently used words in each of Barack Obama and Francisco Barrio's pages:

In [10]:
def top_words(name):
    """
    Get a table of the most frequent words in the given person's wikipedia page.
    """
    row = wiki[wiki['name'] == name]
    # convert a column of dict to columns by apply(pd.Series), then transpose the tabble
    word_count_table = row['word_count'].apply(pd.Series).transpose().reset_index()
    word_count_table.columns= ['word', 'count']
    
    return word_count_table.sort_values(by= 'count', ascending= False)

In [11]:
obama_words = top_words('Barack Obama')
obama_words.head()

,word,count
245,the,40
118,in,30
31,and,21
165,of,18
248,to,14


In [12]:
barrio_words = top_words('Francisco Barrio')
barrio_words.head()

,word,count
204,the,36
147,of,24
23,and,18
102,in,17
95,he,10


Let's extract the list of most frequent words that appear in both Obama's and Barrio's documents. We've so far sorted all words from Obama and Barrio's articles by their word frequencies. We will now use a dataframe operation known as **join**. The **join** operation is very useful when it comes to playing around with data: it lets you combine the content of two tables using a shared column (in this case, the word column). See [the documentation](https://dato.com/products/create/docs/generated/graphlab.SFrame.join.html) for more details.

For instance, running
```
obama_words.join(barrio_words, on='word')
```
will extract the rows from both tables that correspond to the common words.
So to obtain, say, the five common words that appear most often in Obama's article, sort the combined table by the Obama column. 

In [13]:
combined_words = pd.merge(left= obama_words, right = barrio_words, how= 'inner', left_on='word', right_on='word')
combined_words.columns= ['word','Obama','Barrio']
combined_words = combined_words.sort_values(by= 'Obama', ascending=False)
combined_words.head(5)

,word,Obama,Barrio
0,the,40,36
1,in,30,17
2,and,21,18
3,of,18,24
4,to,14,9


**Quiz Question**. Among the words that appear in both Barack Obama and Francisco Barrio, take the 5 that appear most frequently in Obama. How many of the articles in the Wikipedia dataset contain all of those 5 words?

Hint:
* Refer to the previous paragraph for finding the words that appear in both articles. Sort the common words by their frequencies in Obama's article and take the largest five.
* Each word count vector is a Python dictionary. For each word count vector in SFrame, you'd have to check if the set of the 5 common words is a subset of the keys of the word count vector. Complete the function `has_top_words` to accomplish the task.
  - Convert the list of top 5 words into set using the syntax
```
set(common_words)
```
    where `common_words` is a Python list. See [this link](https://docs.python.org/2/library/stdtypes.html#set) if you're curious about Python sets.
  - Extract the list of keys of the word count dictionary by calling the [`keys()` method](https://docs.python.org/2/library/stdtypes.html#dict.keys).
  - Convert the list of keys into a set as well.
  - Use [`issubset()` method](https://docs.python.org/2/library/stdtypes.html#set) to check if all 5 words are among the keys.
* Now apply the `has_top_words` function on every row of the DataFrame.
* Compute the sum of the result column to obtain the number of articles containing all the 5 top words.

In [14]:
common_words = combined_words['word'].head(5).tolist()

def has_top_words(word_count_vector):
    # extract the keys of word_count_vector and convert it to a set
    unique_words = word_count_vector.keys()   # YOUR CODE HERE
    # return True if common_words is a subset of unique_words
    # return False otherwise
    
    return True if set(common_words).issubset(unique_words) else False   
    #return True if set(common_words)<= set(unique_words) else False   

wiki['has_top_words'] = wiki['word_count'].apply(has_top_words)

# use has_top_words column to answer the quiz question
print(sum(wiki['has_top_words'] ) )
wiki['has_top_words'][:5] # YOUR CODE HERE

56066


0     True
1     True
2     True
3     True
4    False
Name: has_top_words, dtype: bool

**Checkpoint**. Check your `has_top_words` function on two random articles:

In [15]:
print 'Output from your function:', has_top_words(wiki.loc[32,'word_count'])
print 'Correct output: True'
print 'Also check the length of unique_words. It should be 167'
print( len( wiki.loc[32,'word_count'].keys()) )

Output from your function: True
Correct output: True
Also check the length of unique_words. It should be 167
167


In [16]:
print 'Output from your function:', has_top_words(wiki.loc[33,'word_count'])
print 'Correct output: False'
print 'Also check the length of unique_words. It should be 188'
print( len( wiki.loc[33,'word_count'].keys()) )

Output from your function: False
Correct output: False
Also check the length of unique_words. It should be 188
188


**Quiz Question**. Measure the pairwise distance between the Wikipedia pages of Barack Obama, George W. Bush, and Joe Biden. Which of the three pairs has the smallest distance?

Hint: To compute the Euclidean distance between two dictionaries, use `graphlab.toolkits.distances.euclidean`. Refer to [this link](https://dato.com/products/create/docs/generated/graphlab.toolkits.distances.euclidean.html) for usage.

In [17]:
OB_text    = DV.transform(wiki.loc[ wiki['name']=='Barack Obama', 'word_count'])
bush_text  = DV.transform(wiki.loc[ wiki['name']=='George W. Bush', 'word_count'])
biden_text =  DV.transform(wiki.loc[ wiki['name']=='Joe Biden', 'word_count'])

from sklearn.metrics.pairwise import pairwise_distances 
print( pairwise_distances ( OB_text , bush_text, metric='euclidean') )
print( pairwise_distances ( OB_text , biden_text, metric='euclidean') )
print( pairwise_distances ( biden_text , bush_text, metric='euclidean') )

[[ 34.39476704]]
[[ 33.07567082]]
[[ 32.75667871]]


**Quiz Question**. Collect all words that appear both in Barack Obama and George W. Bush pages.  Out of those words, find the 10 words that show up most often in Obama's page. 

In [18]:
obama_words = top_words('Barack Obama')
bush_words = top_words('George W. Bush')
combined_words = pd.merge(left= obama_words, right = bush_words, how= 'inner', left_on='word', right_on='word')
combined_words.columns= ['word','Obama','Bush']
combined_words = combined_words.sort_values(by= 'Obama', ascending=False)
combined_words.head(10)

,word,Obama,Bush
0,the,40,39
1,in,30,22
2,and,21,14
3,of,18,14
4,to,14,11
5,his,11,6
6,act,8,3
7,a,7,6
8,he,7,8
9,as,6,6


**Note.** Even though common words are swamping out important subtle differences, commonalities in rarer political words still matter on the margin. This is why politicians are being listed in the query result instead of musicians, for example. In the next subsection, we will introduce a different metric that will place greater emphasis on those rarer words.

## Repeat using precomputed wordcount vectors

In [26]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

In [28]:
word_count = load_sparse_csr('people_wiki_word_count.npz')
tf_idf = load_sparse_csr('people_wiki_tf_idf.npz')

In [198]:
#index_to_word = pd.read_json('people_wiki_map_index_to_word.json')[0]
import json
with open('people_wiki_map_index_to_word.json', 'r') as f: # Reads the list of most frequent words
    index = json.load(f)
map_index_to_word = [str(s) for s in index ]


map_index_to_word

['biennials',
 'murwara',
 'shatzky',
 'woode',
 'damfunk',
 'nualart',
 'potentateswho',
 'missionborn',
 'yeardescribed',
 'theoryhe',
 'vinalop',
 'soestdijk',
 'boncea',
 'spiders',
 'bienniale',
 'woody',
 'trawling',
 'pampoulovawagner',
 'cubasince',
 'laserbased',
 'caner',
 'canes',
 'canet',
 'iaspark',
 'categoriesborn',
 '5982',
 'caney',
 'phosphorushe',
 'yusaf',
 'hhsoffice',
 '5985',
 'gothicmade',
 'caned',
 'sencovici',
 'iguau',
 'storiesin',
 'braziljorge',
 'iguaz',
 'canek',
 'canem',
 'victorialooking',
 'zaikoa',
 'racingteam',
 'sowell',
 'weiskopfs',
 'ninjazz',
 'andersanders',
 'replacer',
 'steppaillen',
 'fjate',
 'heroesin',
 'papanastassiouwasserburgs',
 'voiceactress',
 'fullblooded',
 '723nnings',
 'barcelonasp',
 'genocidekamuhanda',
 'radkes',
 'composertippett',
 'grueling',
 'season1bouyer',
 'wooden',
 'comicborn',
 'virtuosos',
 'remainedin',
 'illinoisrockefellersilvia',
 'woods',
 'woodes',
 'townstephen',
 '598m',
 'immunities',
 'racedue',
 '

In [35]:
model = NearestNeighbors(metric='euclidean', algorithm='brute')
model.fit(word_count)
print wiki[wiki['name'] == 'Barack Obama']

                                              URI          name  \
35817  <http://dbpedia.org/resource/Barack_Obama>  Barack Obama   

                                                    text  \
35817  barack hussein obama ii brk husen bm born augu...   

                                              word_count has_top_words  
35817  {u'represent': 1, u'ending': 1, u'proposition'...          True  


In [78]:
distances, indices = model.kneighbors(word_count[35817], n_neighbors=11) # 1st arg: word count vector
tenNN = pd.concat([pd.Series(indices[0].tolist()), wiki[['name']].iloc[indices[0].tolist()].reset_index().drop('index', axis=1) ,
           pd.Series(distance[0])] , axis=1 )
tenNN.columns = ['id', 'name', 'distance']
tenNN

,id,name,distance
0,35817,Barack Obama,0.000000
1,24478,Joe Biden,1.067634
2,28447,George W. Bush,1.108710
3,35357,Lawrence Summers,1.116294
4,14754,Mitt Romney,1.139642
5,13229,Francisco Barrio,1.147176
6,31423,Walter Mondale,1.148133
7,22745,Wynn Normington Hugh-Jones,1.153384
8,36364,Don Bonker,1.157094
9,9210,Andy Anstett,1.159278


First, run the following cell to obtain the word_count column, which represents the word count vectors in the dictionary form. This way, we can quickly recognize words of great importance.

## TF-IDF to the rescue

Much of the perceived commonalities between Obama and Barrio were due to occurrences of extremely frequent words, such as "the", "and", and "his". So nearest neighbors is recommending plausible results sometimes for the wrong reasons. 

To retrieve articles that are more relevant, we should focus more on rare words that don't happen in every article. **TF-IDF** (term frequency–inverse document frequency) is a feature representation that penalizes words that are too common.  Let's use GraphLab Create's implementation of TF-IDF and repeat the search for the 10 nearest neighbors of Barack Obama:

In [ ]:
tfidf = TfidfTransformer()
dict_vec = DictVectorizer()

tfidf.fit_transform(dict_vec.fit_transform( wiki['word_count']) )

OB_text = tfidf.transform(dict_vec.transform( 
    wiki.loc[wiki['name']== 'Barack Obama', 'word_count']))

In [99]:
model_tf_idf =  NearestNeighbors(metric='euclidean', algorithm='brute')
model_tf_idf.fit(tfidf_vec) 

distance2, index2 = model_tf_idf.kneighbors(X =OB_text, n_neighbors=10) 

pd.concat([wiki[['name']].iloc[index2[0].tolist()].reset_index().drop('index', axis=1) ,
           pd.Series(distance2[0])] , axis=1 )

,name,0
0,Barack Obama,0.000000
1,Joe Biden,1.067634
2,Hillary Rodham Clinton,1.108710
3,Samantha Power,1.116294
4,Eric Stern (politician),1.139642
5,George W. Bush,1.147176
6,John McCain,1.148133
7,Artur Davis,1.153384
8,Henry Waxman,1.157094
9,Jeff Sessions,1.159278


## Repeat using pre-computed tfidf scores to match assignment

In [110]:
OB_index = wiki[wiki['name']== 'Barack Obama'].index.tolist()[0]

model_tf_idf =  NearestNeighbors(metric='euclidean', algorithm='brute')
model_tf_idf.fit(tf_idf) 

distance2, index2 = model_tf_idf.kneighbors(X = tf_idf[OB_index], 
                                            n_neighbors=11) 

pd.concat([wiki[['name']].iloc[index2[0].tolist()].reset_index().drop('index', axis=1) ,
           pd.Series(distance2[0])] , axis=1 )

,name,0
0,Barack Obama,0.000000
1,Phil Schiliro,106.861014
2,Jeff Sessions,108.871674
3,Jesse Lee (politician),109.045698
4,Samantha Power,109.108106
5,Bob Menendez,109.781867
6,Eric Stern (politician),109.957788
7,James A. Guest,110.413889
8,Roland Grossenbacher,110.470609
9,Tulsi Gabbard,110.696998


Let's determine whether this list makes sense.
* With a notable exception of Roland Grossenbacher, the other 8 are all American politicians who are contemporaries of Barack Obama.
* Phil Schiliro, Jesse Lee, Samantha Power, and Eric Stern worked for Obama.

Clearly, the results are more plausible with the use of TF-IDF. Let's take a look at the word vector for Obama and Schilirio's pages. Notice that TF-IDF representation assigns a weight to each word. This weight captures relative importance of that word in the document. Let us sort the words in Obama's article by their TF-IDF weights; we do the same for Schiliro's article as well.

In [200]:
row_index = wiki[wiki['name'] == 'Barack Obama'].index.tolist()[0]
tfidf_data = pd.DataFrame(data = tf_idf[row_index].toarray().transpose(), 
                             index= map_index_to_word ).reset_index()
tfidf_data[tfidf_data[0] >0 ]

,index,0
32093,blobs,10.986495
103658,fisborn,10.986495
120780,polyrhythm,10.986495
397642,sheene,10.293348
409594,afroasiatic,10.293348
452469,bernelli,9.887883
492548,sugdensmith,9.040585
499835,tritium,8.907054
512805,systemwhen,8.421546
526389,disappoint,7.690659


In [156]:
def top_words_tf_idf(name):
    """
    Get a table of the most frequent words in the given person's wikipedia page.
    """
    row_index = wiki[wiki['name'] == name].index.tolist()[0]
    tfidf_data = pd.DataFrame(data = tf_idf[row_index].toarray().transpose(), 
                             index= map_index_to_word ).reset_index()
    tfidf_data.columns=['word', 'weight']
    return tfidf_data[tfidf_data['weight']> 0].sort_values(by= 'weight', ascending= False)

In [157]:
obama_tf_idf = top_words_tf_idf('Barack Obama')
print obama_tf_idf

schiliro_tf_idf = top_words_tf_idf('Phil Schiliro')
print schiliro_tf_idf

                         word     weight
544962                nordahl  43.295653
547084          murderforhire  27.678223
545808                 allows  17.747379
546835    politicianvallarino  14.887061
547685                branwyn  14.722936
544887                 alytus  14.533374
547233                   nuis  13.115933
546139               1960gunn  12.784385
546136             verwaltung  12.784385
547347                alfonse  12.410689
547842           oncehallberg  11.591943
103658                fisborn  10.986495
32093                   blobs  10.986495
120780             polyrhythm  10.986495
397642                 sheene  10.293348
409594            afroasiatic  10.293348
547146              violating  10.164288
452469               bernelli   9.887883
545169               shuntaro   9.431014
545187          segmentsharma   9.419704
547344           2008although   9.319342
547397                   deng   9.077468
492548            sugdensmith   9.040585
545723       dir

Using the **join** operation we learned earlier, try your hands at computing the common words shared by Obama's and Schiliro's articles. Sort the common words by their TF-IDF weights in Obama's document.

In [230]:
combined_words = pd.merge(left= obama_tf_idf , right = schiliro_tf_idf, 
                          how= 'inner', left_on='word', right_on='word')
combined_words.columns= ['word','Obama','Schiliro']
combined_words = combined_words.sort_values(by= 'Obama', ascending=False)
combined_words.head(10)

,word,Obama,Schiliro
0,nordahl,43.295653,9.621256
1,branwyn,14.722936,7.361468
2,alfonse,12.410689,6.205344
3,violating,10.164288,3.388096
4,agua,7.386955,3.693478
5,2009bryant,7.226869,9.033587
6,allreds,6.095386,3.047693
7,2007walkers,5.473201,1.824400
8,kalyana,5.248173,2.624086
9,athenshis,5.107041,3.404694


The first 10 words should say: Obama, law, democratic, Senate, presidential, president, policy, states, office, 2011.

**Quiz Question**. Among the words that appear in both Barack Obama and Phil Schiliro, take the 5 that have largest weights in Obama. How many of the articles in the Wikipedia dataset contain all of those 5 words?

In [221]:
tfidf_df = pd.DataFrame(tf_idf.toarray()) 
tfidf_df.columns= map_index_to_word

In [226]:
print(common_words)
mat = tfidf_df[common_words]
mat0= mat[mat[common_words[0]] >0]
mat1= mat0[mat0[common_words[1]] >0]
mat2= mat1[mat1[common_words[2]] >0]
mat3= mat2[mat2[common_words[3]] >0]
mat4= mat3[mat3[common_words[4]] >0]
mat4

['nordahl', 'branwyn', 'alfonse', 'violating', 'agua']


,nordahl,branwyn,alfonse,violating,agua
3384,9.621256,4.907645,6.205344,3.388096,7.386955
6896,4.810628,2.453823,6.205344,3.388096,7.386955
7914,9.621256,7.361468,6.205344,3.388096,3.693478
7950,4.810628,7.361468,6.205344,6.776192,3.693478
8063,4.810628,2.453823,6.205344,10.164288,7.386955
11517,9.621256,9.815291,6.205344,3.388096,3.693478
14754,4.810628,2.453823,3.102672,3.388096,18.467389
15795,4.810628,2.453823,3.102672,6.776192,3.693478
24478,19.242512,2.453823,9.308017,10.164288,7.386955
35817,43.295653,14.722936,12.410689,10.164288,7.386955


In [229]:
len(mat4)

14

In [228]:
# names who have these top words
wiki.loc[mat4.index.tolist(),'name']

3384                 Caroline Kennedy
6896     Jonathan Singer (journalist)
7914                    Phil Schiliro
7950                 Elizabeth Warren
8063                     Sarah Sewall
11517                    Louis Susman
14754                     Mitt Romney
15795                   Larry Kilgore
24478                       Joe Biden
35817                    Barack Obama
45068                  Douglas Schoen
47303                      John Kerry
55880                      Sheila Nix
57108          Hillary Rodham Clinton
Name: name, dtype: object

Notice the huge difference in this calculation using TF-IDF scores instead  of raw word counts. We've eliminated noise arising from extremely common words.

## Choosing metrics

You may wonder why Joe Biden, Obama's running mate in two presidential elections, is missing from the query results of `model_tf_idf`. Let's find out why. First, compute the distance between TF-IDF features of Obama and Biden.

**Quiz Question**. Compute the Euclidean distance between TF-IDF features of Obama and Biden. Hint: When using Boolean filter in SFrame/SArray, take the index 0 to access the first match.

In [218]:
obama_tf_idf = top_words_tf_idf('Barack Obama')
biden_tf_idf = top_words_tf_idf('Joe Biden')

combined_words = pd.merge(left= obama_tf_idf , right = biden_tf_idf, 
                          how= 'outer', left_on='word', right_on='word')
combined_words.columns= ['word','Obama','Biden']
combined_words = combined_words.fillna(0)
combined_words.head()
combined_words

,word,Obama,Biden
0,nordahl,43.295653,19.242512
1,murderforhire,27.678223,17.298889
2,allows,17.747379,4.436845
3,politicianvallarino,14.887061,7.443530
4,branwyn,14.722936,2.453823
5,alytus,14.533374,0.000000
6,nuis,13.115933,3.278983
7,1960gunn,12.784385,0.000000
8,verwaltung,12.784385,0.000000
9,alfonse,12.410689,9.308017


In [219]:
from sklearn.metrics.pairwise import pairwise_distances 
print( pairwise_distances ( combined_words['Obama'].values.reshape(1,-1), 
                           combined_words['Biden'].values.reshape(1,-1),
                           metric='euclidean') )[0]

[ 123.29745601]


The distance is larger than the distances we found for the 10 nearest neighbors, which we repeat here for readability:

In [231]:
OB_index = wiki[wiki['name']== 'Barack Obama'].index.tolist()[0]

model_tf_idf =  NearestNeighbors(metric='euclidean', algorithm='brute')
model_tf_idf.fit(tf_idf) 

distance2, index2 = model_tf_idf.kneighbors(X = tf_idf[OB_index], 
                                            n_neighbors=11) 

pd.concat([wiki[['name']].iloc[index2[0].tolist()].reset_index().drop('index', axis=1) ,
           pd.Series(distance2[0])] , axis=1 )

,name,0
0,Barack Obama,0.000000
1,Phil Schiliro,106.861014
2,Jeff Sessions,108.871674
3,Jesse Lee (politician),109.045698
4,Samantha Power,109.108106
5,Bob Menendez,109.781867
6,Eric Stern (politician),109.957788
7,James A. Guest,110.413889
8,Roland Grossenbacher,110.470609
9,Tulsi Gabbard,110.696998


But one may wonder, is Biden's article that different from Obama's, more so than, say, Schiliro's? It turns out that, when we compute nearest neighbors using the Euclidean distances, we unwittingly favor short articles over long ones. Let us compute the length of each Wikipedia document, and examine the document lengths for the 100 nearest neighbors to Obama's page.

In [234]:
def compute_length(row):
    return len(row['text'].split(' '))

wiki['length'] = wiki.apply(compute_length, axis=1) 

In [235]:
nearest_neighbors_euclidean = model_tf_idf.query(wiki[wiki['name'] == 'Barack Obama'], label='name', k=100)
nearest_neighbors_euclidean = nearest_neighbors_euclidean.join(wiki[['name', 'length']], on={'reference_label':'name'})

AttributeError: 'NearestNeighbors' object has no attribute 'query'

In [ ]:
nearest_neighbors_euclidean.sort('rank')

To see how these document lengths compare to the lengths of other documents in the corpus, let's make a histogram of the document lengths of Obama's 100 nearest neighbors and compare to a histogram of document lengths for all documents.

In [ ]:
plt.figure(figsize=(10.5,4.5))
plt.hist(wiki['length'], 50, color='k', edgecolor='None', histtype='stepfilled', normed=True,
         label='Entire Wikipedia', zorder=3, alpha=0.8)
plt.hist(nearest_neighbors_euclidean['length'], 50, color='r', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (Euclidean)', zorder=10, alpha=0.8)
plt.axvline(x=wiki['length'][wiki['name'] == 'Barack Obama'][0], color='k', linestyle='--', linewidth=4,
           label='Length of Barack Obama', zorder=2)
plt.axvline(x=wiki['length'][wiki['name'] == 'Joe Biden'][0], color='g', linestyle='--', linewidth=4,
           label='Length of Joe Biden', zorder=1)
plt.axis([0, 1000, 0, 0.04])

plt.legend(loc='best', prop={'size':15})
plt.title('Distribution of document length')
plt.xlabel('# of words')
plt.ylabel('Percentage')
plt.rcParams.update({'font.size':16})
plt.tight_layout()

Relative to the rest of Wikipedia, nearest neighbors of Obama are overwhemingly short, most of them being shorter than 300 words. The bias towards short articles is not appropriate in this application as there is really no reason to  favor short articles over long articles (they are all Wikipedia articles, after all). Many of the Wikipedia articles are 300 words or more, and both Obama and Biden are over 300 words long.

**Note**: For the interest of computation time, the dataset given here contains _excerpts_ of the articles rather than full text. For instance, the actual Wikipedia article about Obama is around 25000 words. Do not be surprised by the low numbers shown in the histogram.

**Note:** Both word-count features and TF-IDF are proportional to word frequencies. While TF-IDF penalizes very common words, longer articles tend to have longer TF-IDF vectors simply because they have more words in them.

To remove this bias, we turn to **cosine distances**:
$$
d(\mathbf{x},\mathbf{y}) = 1 - \frac{\mathbf{x}^T\mathbf{y}}{\|\mathbf{x}\| \|\mathbf{y}\|}
$$
Cosine distances let us compare word distributions of two articles of varying lengths.

Let us train a new nearest neighbor model, this time with cosine distances.  We then repeat the search for Obama's 100 nearest neighbors.

In [ ]:
model2_tf_idf = graphlab.nearest_neighbors.create(wiki, label='name', features=['tf_idf'],
                                                  method='brute_force', distance='cosine')

In [ ]:
nearest_neighbors_cosine = model2_tf_idf.query(wiki[wiki['name'] == 'Barack Obama'], label='name', k=100)
nearest_neighbors_cosine = nearest_neighbors_cosine.join(wiki[['name', 'length']], on={'reference_label':'name'})

In [ ]:
nearest_neighbors_cosine.sort('rank')

From a glance at the above table, things look better.  For example, we now see Joe Biden as Barack Obama's nearest neighbor!  We also see Hillary Clinton on the list.  This list looks even more plausible as nearest neighbors of Barack Obama.

Let's make a plot to better visualize the effect of having used cosine distance in place of Euclidean on our TF-IDF vectors.

In [ ]:
plt.figure(figsize=(10.5,4.5))
plt.figure(figsize=(10.5,4.5))
plt.hist(wiki['length'], 50, color='k', edgecolor='None', histtype='stepfilled', normed=True,
         label='Entire Wikipedia', zorder=3, alpha=0.8)
plt.hist(nearest_neighbors_euclidean['length'], 50, color='r', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (Euclidean)', zorder=10, alpha=0.8)
plt.hist(nearest_neighbors_cosine['length'], 50, color='b', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (cosine)', zorder=11, alpha=0.8)
plt.axvline(x=wiki['length'][wiki['name'] == 'Barack Obama'][0], color='k', linestyle='--', linewidth=4,
           label='Length of Barack Obama', zorder=2)
plt.axvline(x=wiki['length'][wiki['name'] == 'Joe Biden'][0], color='g', linestyle='--', linewidth=4,
           label='Length of Joe Biden', zorder=1)
plt.axis([0, 1000, 0, 0.04])
plt.legend(loc='best', prop={'size':15})
plt.title('Distribution of document length')
plt.xlabel('# of words')
plt.ylabel('Percentage')
plt.rcParams.update({'font.size': 16})
plt.tight_layout()

Indeed, the 100 nearest neighbors using cosine distance provide a sampling across the range of document lengths, rather than just short articles like Euclidean distance provided.

**Moral of the story**: In deciding the features and distance measures, check if they produce results that make sense for your particular application.

# Problem with cosine distances: tweets vs. long articles

Happily ever after? Not so fast. Cosine distances ignore all document lengths, which may be great in certain situations but not in others. For instance, consider the following (admittedly contrived) example.

```
+--------------------------------------------------------+
|                                             +--------+ |
|  One that shall not be named                | Follow | |
|  @username                                  +--------+ |
|                                                        |
|  Democratic governments control law in response to     |
|  popular act.                                          |
|                                                        |
|  8:05 AM - 16 May 2016                                 |
|                                                        |
|  Reply   Retweet (1,332)   Like (300)                  |
|                                                        |
+--------------------------------------------------------+
```

How similar is this tweet to Barack Obama's Wikipedia article? Let's transform the tweet into TF-IDF features, using an encoder fit to the Wikipedia dataset.  (That is, let's treat this tweet as an article in our Wikipedia dataset and see what happens.)

In [ ]:
sf = graphlab.SFrame({'text': ['democratic governments control law in response to popular act']})
sf['word_count'] = graphlab.text_analytics.count_words(sf['text'])

encoder = graphlab.feature_engineering.TFIDF(features=['word_count'], output_column_prefix='tf_idf')
encoder.fit(wiki)
sf = encoder.transform(sf)
sf

Let's look at the TF-IDF vectors for this tweet and for Barack Obama's Wikipedia entry, just to visually see their differences.

In [ ]:
tweet_tf_idf = sf[0]['tf_idf.word_count']
tweet_tf_idf

In [ ]:
obama = wiki[wiki['name'] == 'Barack Obama']
obama

Now, compute the cosine distance between the Barack Obama article and this tweet:

In [ ]:
obama_tf_idf = obama[0]['tf_idf']
graphlab.toolkits.distances.cosine(obama_tf_idf, tweet_tf_idf)

Let's compare this distance to the distance between the Barack Obama article and all of its Wikipedia 10 nearest neighbors:

In [ ]:
model2_tf_idf.query(obama, label='name', k=10)

With cosine distances, the tweet is "nearer" to Barack Obama than everyone else, except for Joe Biden!  This probably is not something we want. If someone is reading the Barack Obama Wikipedia page, would you want to recommend they read this tweet? Ignoring article lengths completely resulted in nonsensical results. In practice, it is common to enforce maximum or minimum document lengths. After all, when someone is reading a long article from _The Atlantic_, you wouldn't recommend him/her a tweet.